In [8]:
!nvidia-smi

Wed Oct  4 11:47:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8               6W /  60W |    719MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
HOME=os.getcwd()
print(HOME)

/home/navid/deep_learning/fire_detection_yolo


In [ ]:
!pip install ultralytics



In [4]:
from ultralytics import YOLO
from IPython.display import display,Image

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="AqGT2fWfDNoNlYkDREdK")
project = rf.workspace("navid-a-m-mz33r").project("fire-detection-qggmx")
dataset = project.version(1).download("yolov8")


In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train6/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train6/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train6/val_batch0_pred.jpg', width=600)

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train6/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/weights/best.pt conf=0.25 source="/home/navid/deep_learning/fire_detection_yolo/images/test1.png" save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [5]:
%cd {HOME}/videos
import cv2
# Open video file
video_path = "/home/navid/deep_learning/fire_detection_yolo/videos/fire2.mp4"
print(video_path)
cap = cv2.VideoCapture(video_path)

# Load YOLO model from the specified path
model = YOLO("/home/navid/deep_learning/fire_detection_yolo/weights/best.pt","v8")
print(model)


/home/navid/deep_learning/fire_detection_yolo/videos
/home/navid/deep_learning/fire_detection_yolo/videos/fire2.mp4
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1

In [6]:

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec (use 'XVID' for .avi)
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform fire detection on the frame
    results = model.predict(frame, conf=0.25)
    print(results)
    # Access detected boxes
    boxes = results[0].cpu().numpy()
    print("BOXES: ",boxes)
    # Draw bounding boxes on the frame (if fire is detected)
    try:
        for box in boxes:
            cv2.rectangle(frame, 
                          (int(box[0]), int(box[1])), 
                          (int(box[2]), int(box[3])), 
                          (0, 255, 0), 
                          2)

    except Exception as e:
        print("Fire not detected: ",e)
    # Write the modified frame to the output video
    out.write(frame)

    # Display the frame with OpenCV
    cv2.imshow("Fire Detection", frame)
    cv2.waitKey()
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release video capture, release VideoWriter, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()



0: 800x480 (no detections), 146.4ms
Speed: 10.5ms preprocess, 146.4ms inference, 1.8ms postprocess per image at shape (1, 3, 800, 480)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'fire'}
orig_img: array([[[159, 175, 194],
        [159, 175, 194],
        [159, 175, 194],
        ...,
        [101, 115, 154],
        [103, 117, 156],
        [104, 118, 157]],

       [[159, 175, 194],
        [159, 175, 194],
        [159, 175, 194],
        ...,
        [103, 117, 156],
        [104, 118, 157],
        [105, 119, 158]],

       [[159, 175, 194],
        [159, 175, 194],
        [159, 175, 194],
        ...,
        [103, 117, 156],
        [101, 115, 154],
        [100, 114, 153]],

       ...,

       [[ 30,  20,  15],
        [ 30,  20,  15],
        [ 32,  22,  17],
        ...,
        [ 73,  74,  97],
        [ 76,  77, 100],
        [ 78,  79, 102]],

       [[ 30,  20,  15],
        [ 30,  20,  15],
        [ 32,  22,  17],
        ...,
        [ 74,  75,  98],
        [ 77,  78, 101],
        [ 79,  80, 103]]

error: OpenCV(4.8.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [7]:
!yolo task=detect mode=predict model="/home/navid/deep_learning/fire_detection_yolo/weights/best.pt" conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

Ultralytics YOLOv8.0.191 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3902MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

100%|████████████████████████████████████████| 104k/104k [00:00<00:00, 1.65MB/s]
image 1/1 /home/navid/deep_learning/fire_detection_yolo/videos/dog.jpeg: 800x480 (no detections), 130.8ms
Speed: 2.9ms preprocess, 130.8ms inference, 5.9ms postprocess per image at shape (1, 3, 800, 480)
Results saved to runs/detect/predict30
💡 Learn more at https://docs.ultralytics.com/modes/predict
